This notebook is an example on how to use this framework. Random Search has been used as NAS with image folder as an input example. 

The first thing to do is to choose the input data format. In this notebook we will choose the numpy images. You have to import the correct python files in the `data_pipeline.py`. 

In [11]:
from torchvision import transforms
from torchvision.transforms.transforms import Compose
from framework.numpy_image import Image_Data

Depending on the augment_style, you may want to add something else that not in the `data_augmentation` function. Let's add a custom augment style.

In [12]:
class Data_Pipeline:
    def __init__(self, data_info, augment_style, BATCHSIZE):

        self.data_info = data_info
        self.augment_style = augment_style
        self.batchsize = BATCHSIZE

        (
            self.train_loader,
            self.valid_loader,
            self.test_loader,
            self.n_classes,
            self.n_in,
        ) = self.data_process()

    def data_augmentation(self, augment_style: str):

        crop_size = 224

        # Setup data transforms:
        flip = transforms.RandomHorizontalFlip()
        resize = transforms.Resize((crop_size, crop_size))
        tensor_transform = transforms.ToTensor()
        crop = transforms.RandomCrop(crop_size)
        colorjitter = transforms.ColorJitter(brightness=0.6, contrast=0.6, saturation=0.6, hue=0.5)
        shear = transforms.RandomAffine(degrees=0, translate=None, scale=None, shear=20) # 20 degrees random shear

        if augment_style == "default":
            train_transforms = transforms.Compose([resize, tensor_transform])

        elif augment_style == "flip":
            train_transforms = transforms.Compose([resize, flip, tensor_transform])

        elif augment_style =="custom":
            train_transforms = transforms.Compose([resize, flip, shear, tensor_transform])

        test_transforms = transforms.Compose(
            [transforms.Resize((crop_size, crop_size)), transforms.ToTensor()]
        )
        return train_transforms, test_transforms

    def data_process(self):

        train_transforms, test_transforms = self.data_augmentation(self.augment_style)

        (
            train_loader,
            valid_loader,
            test_loader,
            n_classes,
            n_in,
        ) = Image_Data.get_data(
            self.data_info, self.batchsize, train_transforms, test_transforms
        )

        return train_loader, valid_loader, test_loader, n_classes, n_in

In the `main_framework.py`, you need to specify some args in the `data_process`. But before that, let's import some library and class and declare some initial variable.

#### Import

In [13]:
from pathlib import Path
import time

from framework.time_counter import Clock, show_time

# from data_pipeline import Data_Pipeline
from framework.train_pipeline import Train_pipeline

#### Time

In [14]:
total_runtime_hours = 2
total_runtime_seconds = total_runtime_hours * 60 * 60
start_time = time.time()
runclock = Clock(total_runtime_seconds)

#### Data Process
###### data info is depending on data type used. 
###### for image_download, data_info can be MNIST or fashion_mnist or..
###### for image_folder, data_info is the path to the the image folder
###### for numpy_image, , data_info is the path to the the numpy files

Assign the `data_info` to the path of the numpy images.
Assign the `augment_style` to custom and choose the batch size.

In [15]:
print("=== Processing Data ===")
print("  Estimated time left:", show_time(runclock.check()))

data_process = Data_Pipeline(
        data_info=f"{Path.home()}/Data/NAS/Nas_data/devel_dataset_0",
        augment_style="custom",
        BATCHSIZE=28,
    )

=== Processing Data ===
  Estimated time left: 1h,59m,59s
/home/nikkhadijah/Data/NAS/Nas_data/devel_dataset_0


#### NAS -- Random Search

##### Import

In [16]:
import torch
import torch.nn as nn
import random

##### ConvNet as model

In [17]:
class ConvNet(nn.Module):
    def __init__(self, num_classes, dropout, rectify, maxpool, enc_sizes, n_in, n_out):
        super().__init__()

        def my_conv_block(in_f, out_f):
            return nn.Sequential(
                nn.Conv2d(in_f, out_f, kernel_size=3, padding=1), rectify, maxpool
            )

        self.features1 = nn.Sequential(
            nn.Conv2d(n_in, 64, kernel_size=(7, 7), stride=1, padding=3),
            rectify,
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.enc_sizes = [n_out, *enc_sizes]

        convolution_blocks = [
            my_conv_block(in_f, out_f)
            for in_f, out_f in zip(self.enc_sizes, self.enc_sizes[1:])
        ]

        self.features_blocks = nn.Sequential(*convolution_blocks)

        self.features2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            rectify,
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            rectify,
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            rectify,
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            rectify,
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:

        x = self.features1(x)
        x = self.features_blocks(x)
        x = self.features2(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

##### Random Search

In [18]:
class NAS:
    def __init__(self):
        pass

    def search(self, n_classes, n_in, n_out, dropout):

        rectifier_n = random.randint(0, 2)
        if rectifier_n == 0:
            rectify = nn.ReLU(inplace=True)
        elif rectifier_n == 1:
            rectify = nn.PReLU()
        else:
            rectify = nn.LeakyReLU(inplace=True)

        maxpool_n = 1
        if maxpool_n == 0:
            conv_block = random.randint(1, 2)
            maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        else:
            conv_block = random.randint(1, 5)
            maxpool = nn.Identity()

        enc_sizes = [n_out] * conv_block

        model = ConvNet(n_classes, dropout, rectify, maxpool, enc_sizes, n_in, n_out)
        optimizer_name = "Adam"
        lr = 0.00001
        return model, optimizer_name, lr



##### NAS pipeline

In [19]:
class NAS_pipeline:
    def __init__(self, n_classes, n_in):
        self.n_out = 64
        self.dropout = 0.5
        self.n_classes = n_classes
        self.n_in = n_in

        self.model, self.optimizer_name, self.lr = self.nas_pipeline()

    def nas_pipeline(self):

        nas_algorithm = NAS()
        model, optimizer_name, lr = nas_algorithm.search(
            self.n_classes, self.n_in, self.n_out, self.dropout
        )

        return model, optimizer_name, lr

#### Function call

In [20]:
print("=== Performing NAS ===")
print("  Estimated time left:", show_time(runclock.check()))
model = NAS_pipeline(data_process.n_classes, data_process.n_in)

=== Performing NAS ===
  Estimated time left: 1h,59m,59s


#### Training

In [21]:
print("=== Training ===")
print("  Estimated time left:", show_time(runclock.check()))
train = Train_pipeline(model, data_process.train_loader, data_process.valid_loader)

=== Training ===
  Estimated time left: 1h,59m,59s
	Epoch   1/1   | Train Acc:   5.41% | Valid Acc:   5.62% |


#### Results

In [22]:
print("=== Predicting ===")
print("  Estimated time left:", show_time(runclock.check()))
test_results = train.test(data_process.test_loader)

results = {}
results['time_left'] = show_time(runclock.check())
results['total_time'] = show_time(time.time() - start_time)
results['accuracy'] = test_results[0]
results['no_parameters'] = test_results[1]

#     # save to pickel or csv
print(results)

=== Predicting ===
  Estimated time left: 1h,55m,34s
{'time_left': '1h,55m,16s', 'total_time': '4m,43s', 'accuracy': 0.03381883479648846, 'no_parameters': 55031508}
